In [1]:
import os
os.chdir('..')

In [2]:
from copy import copy

from kernel.type import TFun, BoolType, NatType
from kernel.term import Var, Implies, Forall, BoolVars, Inst, And, Or, Exists
from kernel.proofterm import ProofTerm, TacticException
from kernel.thm import Thm
from kernel import theory
from logic import basic
from logic.tactic import Tactic
from logic.matcher import MatchException, first_order_match
from syntax.settings import settings

basic.load_theory('nat')
settings.unicode = True

## Tactics

So far our proofs are constructed mostly in the bottom-up fashion. That is, the proof term is built up by first building its components, then composing these components together. Often, a different approach, constructing a proof top-down, is more appropriate. In this approach, a goal is reduced to one or more simpler subgoals, which are further reduced, until there is no subgoal remaining. The system of *tactics* permits this kind of reasoning.

A tactic is a function that takes a theorem as input, and returns a proof term proving that theorem. The returned proof term may contain gaps (constructed by the rule `sorry`), indicating remaining goals to be proved. Hence, a tactic can also be viewed as transforming a goal to a list of subgoals. Like conversions, we can compose tactics together to form more complex tactics using *tacticals*.

Tactics is represented in holpy as the class `Tactic`. New tactics are implemented by inheriting from this class. A tactic can take additional arguments through its `__init__` function. The `get_proof_term` function takes a goal (of type `Thm`) as argument, and should return a proof term whose conclusion is the goal. The returned proof term may contain gaps.

Let's begin with some simple examples of tactics. The `intro_imp_tac` tactic takes an implication goal, and moves one assumption to the hypothesis. It applies the following proof rule in the reverse direction:

$$ \frac{\Gamma, A \vdash C}{\Gamma \vdash A\to C} $$

In [3]:
class intro_imp_tac(Tactic):
    def get_proof_term(self, goal):
        if not goal.prop.is_implies():
            raise TacticException('intro_imp: goal is not implies.')

        A, C = goal.prop.args
        new_goal = ProofTerm.sorry(Thm(list(goal.hyps) + [A], C))
        return new_goal.implies_intr(A)

We can test the tactic directly on a theorem (note the printing function for a proof term also prints the remaining goals to be proved).

In [4]:
x = Var('x', NatType)
P = Var('P', TFun(NatType, BoolType))
Q = Var('Q', TFun(NatType, BoolType))
A = Var('A', BoolType)

th = Thm([A], Implies(P(x), Q(x)))
print('goal:', th)

pt = intro_imp_tac().get_proof_term(th)
print('pt:', pt)

print('\nProof:')
print(pt.export())

goal: A ⊢ P x ⟶ Q x
pt: ProofTerm(A ⊢ P x ⟶ Q x)
Gaps: A, P x ⊢ Q x

Proof:
0: A, P x ⊢ Q x by sorry
1: A ⊢ P x ⟶ Q x by implies_intr P x from 0


The `intro_forall_tac` takes an forall goal, and introduces a new variable. If a variable name is provided it is used. Otherwise the name provided by the abstraction is used. It applies the following proof rule in the reverse direction:

$$ \frac{\Gamma \vdash P(a)}{\Gamma \vdash \forall a.\,P(a)}\hbox{ ($a$ does not occur in $\Gamma$)} $$

In [5]:
class intro_forall_tac(Tactic):
    def __init__(self, var_name=None):
        self.var_name = var_name
        
    def get_proof_term(self, goal):
        if not goal.prop.is_forall():
            raise TacticException('intro_forall: goal is not forall')

        v, body = goal.prop.arg.dest_abs(self.var_name)
        new_goal = ProofTerm.sorry(Thm(goal.hyps, body))
        return new_goal.forall_intr(v)

In [6]:
th = Thm([A], Forall(x, Implies(P(x), Q(x))))
print('goal:', th)

pt = intro_forall_tac().get_proof_term(th)
print('pt:', pt)

print('\nProof:')
print(pt.export())

goal: A ⊢ ∀x. P x ⟶ Q x
pt: ProofTerm(A ⊢ ∀x. P x ⟶ Q x)
Gaps: A ⊢ P x ⟶ Q x

Proof:
0: A ⊢ P x ⟶ Q x by sorry
1: A ⊢ ∀x. P x ⟶ Q x by forall_intr x from 0


A tactic can also be applied on a proof term, through the proof term's `tac` method. The method applies the tactic to the *first* subgoal of the proof term.

In [7]:
pt = ProofTerm.sorry(Thm([A], Implies(P(x), Q(x))))
print('pt:', pt)

pt2 = pt.tac(intro_imp_tac())
print('pt2:', pt2)

pt: ProofTerm(A ⊢ P x ⟶ Q x)
Gaps: A ⊢ P x ⟶ Q x
pt2: ProofTerm(A ⊢ P x ⟶ Q x)
Gaps: A, P x ⊢ Q x


Several tactics can be applied in sequence using the `tacs` method:

In [8]:
pt = ProofTerm.sorry(Thm([A], Forall(x, Implies(P(x), Q(x)))))
print('pt:', pt)

pt2 = pt.tacs(intro_forall_tac('a'), intro_imp_tac())
print('pt2:', pt2)

pt: ProofTerm(A ⊢ ∀x. P x ⟶ Q x)
Gaps: A ⊢ ∀x. P x ⟶ Q x
pt2: ProofTerm(A ⊢ ∀a. P a ⟶ Q a)
Gaps: A, P a ⊢ Q a


## Tacticals

A tactical can be used to compose together tactics to form more complex tactics. This works in a way similar to conversion combinators. In holpy, a tactical is also implemented by inheriting from class `Tactic`, taking the component tactics as arguments to the `__init__` function. We first demonstrate this with the `then_tac` tactical:

In [9]:
class then_tac(Tactic):
    def __init__(self, tac1, tac2):
        self.tac1 = tac1
        self.tac2 = tac2
        
    def get_proof_term(self, goal):
        return ProofTerm.sorry(goal).tacs(self.tac1, self.tac2)

Use `tacs` to apply a sequence of tactics is the same as using `tac` to apply the `then_tac` of these tactics. For example:

In [10]:
pt = ProofTerm.sorry(Thm([A], Forall(x, Implies(P(x), Q(x)))))
print('pt:', pt)

pt2 = pt.tac(then_tac(intro_forall_tac('a'), intro_imp_tac()))
print('pt2:', pt2)

pt: ProofTerm(A ⊢ ∀x. P x ⟶ Q x)
Gaps: A ⊢ ∀x. P x ⟶ Q x
pt2: ProofTerm(A ⊢ ∀a. P a ⟶ Q a)
Gaps: A, P a ⊢ Q a


The tactical `else_tac` tries to apply the first tactic, and if fails, the second tactics:

In [11]:
class else_tac(Tactic):
    def __init__(self, tac1, tac2):
        self.tac1 = tac1
        self.tac2 = tac2
        
    def get_proof_term(self, goal):
        try:
            return self.tac1.get_proof_term(goal)
        except TacticException:
            return self.tac2.get_proof_term(goal)

For example, we can form the tactic `intro_one`, which introduces either an implication or a forall quantification:

In [12]:
intro_one_tac = else_tac(intro_imp_tac(), intro_forall_tac())

print(intro_one_tac.get_proof_term(Thm([A], Implies(P(x), Q(x)))))
print(intro_one_tac.get_proof_term(Thm([A], Forall(x, Implies(P(x), Q(x))))))

ProofTerm(A ⊢ P x ⟶ Q x)
Gaps: A, P x ⊢ Q x
ProofTerm(A ⊢ ∀x. P x ⟶ Q x)
Gaps: A ⊢ P x ⟶ Q x


The tactical `repeat_tac` repeatedly applies some tactic, until `TacticException` is raised:

In [13]:
class repeat_tac(Tactic):
    def __init__(self, tac):
        self.tac = tac
        
    def get_proof_term(self, goal):
        pt = ProofTerm.sorry(goal)
        while True:
            try:
                pt = pt.tac(self.tac)
            except TacticException:
                break
        return pt

The tactic `intros_tac` repeatedly introduces implication and forall quantification:

In [14]:
intros_tac = repeat_tac(else_tac(intro_imp_tac(), intro_forall_tac()))

print(intros_tac.get_proof_term(Thm([A], Forall(x, Implies(P(x), Q(x))))))

ProofTerm(A ⊢ ∀x. P x ⟶ Q x)
Gaps: A, P x ⊢ Q x


Implication and forall quantification can also alternate:

In [15]:
x = Var('x', NatType)
y = Var('y', NatType)
P = Var('P', TFun(NatType, BoolType))
Q = Var('Q', TFun(NatType, BoolType))
R = Var('R', TFun(NatType, NatType, BoolType))

goal = Forall(x, Implies(P(x), Forall(y, Implies(Q(y), R(x, y)))))
print('goal:', goal)
print(intros_tac.get_proof_term(Thm([], goal)))

goal: ∀x. P x ⟶ (∀y. Q y ⟶ R x y)
ProofTerm(⊢ ∀x. P x ⟶ (∀y. Q y ⟶ R x y))
Gaps: P x, Q y ⊢ R x y


## Applying theorems

We now discuss tactics for applying a single theorem. The tactic `rule_tac` applies a theorem in the backward direction. It matches the current goal with the conclusion of the theorem. If there is a match, it replaces the goal with the assumption of the theorem. The tactics takes an additional optional parameter specifying part of the instantiation. This tactic can be implemented as follows:

In [16]:
class rule_tac(Tactic):
    def __init__(self, th_name, *, inst=None):
        self.th_name = th_name
        if inst is None:
            inst = Inst()
        self.inst = inst
        
    def get_proof_term(self, goal):
        th = theory.get_theorem(self.th_name)        
        try:
            inst = first_order_match(th.concl, goal.prop, self.inst)
        except MatchException:
            raise TacticException('rule: matching failed')
            
        if any(v.name not in inst for v in th.prop.get_svars()):
            raise TacticException('rule: not all variables are matched')
            
        pt = ProofTerm.theorem(self.th_name).substitution(inst)
        for assum in pt.assums:
            pt = pt.implies_elim(ProofTerm.sorry(Thm(goal.hyps, assum)))
        return pt

We test this tactic on some simple theorems:

In [17]:
P, Q = BoolVars('P Q')
pt = ProofTerm.sorry(Thm([And(P, Q)], And(Q, P)))
print('pt:', pt)

pt2 = pt.tac(rule_tac('conjI'))
print('pt2:', pt2)

pt: ProofTerm(P ∧ Q ⊢ Q ∧ P)
Gaps: P ∧ Q ⊢ Q ∧ P
pt2: ProofTerm(P ∧ Q ⊢ Q ∧ P)
Gaps: P ∧ Q ⊢ Q
      P ∧ Q ⊢ P


We see that the tactic reduces the goal $Q\wedge P$ into two goals $Q$ and $P$, with the same hypothesis. We can try to use this theorem with `conjD1` and `conjD2` to prove this theorem completely, this time by providing some instantiation.

In [18]:
print(pt.tacs(rule_tac('conjI'), rule_tac('conjD2', inst=Inst(A=P))))

ProofTerm(P ∧ Q ⊢ Q ∧ P)
Gaps: P ∧ Q ⊢ P ∧ Q
      P ∧ Q ⊢ P


We see that we will need another tactic that solves a goal when it already appears in the assumption.

In [19]:
class assumption(Tactic):
    def get_proof_term(self, goal):
        if not goal.prop in goal.hyps:
            raise TacticException('assumption: prop does not appear in hyps')
        
        return ProofTerm.assume(goal.prop)

The reason this works is because the proof term returned by the tactic does not need to exactly match the goal, as long as it has the same consequent as the goal, and the antecedent is a subset of the antecedent of the goal.

In [20]:
print(pt.tacs(
    rule_tac('conjI'),
    rule_tac('conjD2', inst=Inst(A=P)), assumption(),
    rule_tac('conjD1', inst=Inst(B=Q)), assumption()
))

ProofTerm(P ∧ Q ⊢ Q ∧ P)


## Elimination

Theorems like `conjI` is well-suited for application by matching only the conclusion. However, many other theorems are well-suited to application by matching the conclusion as well as the first assumption. A representative example is `disjE`:

$$ A \vee B \to (A \to C) \to (B \to C) \to C $$

Here the conclusion is arbitrary, and the key is matching the assumption $A\vee B$, replacing the goal $C$ by two subgoals $A\to C$ and $B\to C$. This corresponds to case analysis on the disjunction $A\vee B$. The `elim` tactic implements this function. In addition to the theorem name, it takes an optional parameter specifying the term $A\vee B$. If the parameter is not given, it finds the first term in the antecedent of the goal matching this pattern (it should be clear that relying on this feature is not very robust).

In [21]:
class elim_tac(Tactic):
    def __init__(self, th_name, *, cond=None, inst=None):
        self.th_name = th_name
        if inst is None:
            inst = Inst()
        self.inst = inst
        self.cond = cond
        
    def get_proof_term(self, goal):
        th = theory.get_theorem(self.th_name)

        assum = th.assums[0]
        cond = self.cond
        if cond is None:
            # Find cond by matching with goal.hyps one by one
            for hyp in goal.hyps:
                try:
                    inst = first_order_match(th.assums[0], hyp, self.inst)
                    cond = hyp
                    break
                except MatchException:
                    pass
        
        if cond is None:
            raise TacticException('elim: cannot match assumption')

        try:
            inst = first_order_match(th.concl, goal.prop, inst)
        except MatchException:
            raise TacticException('elim: matching failed')

        if any(v.name not in inst for v in th.prop.get_svars()):
            raise TacticException('elim: not all variables are matched')
            
        pt = ProofTerm.theorem(self.th_name).substitution(inst)
        pt = pt.implies_elim(ProofTerm.assume(cond))
        for assum in pt.assums:
            pt = pt.implies_elim(ProofTerm.sorry(Thm(goal.hyps, assum)))
        return pt

Let's test this tactic with `disjE`:

In [22]:
P = Var('P', BoolType)
Q = Var('Q', BoolType)
pt = ProofTerm.sorry(Thm([Or(P, Q)], Or(Q, P)))
print('pt:', pt)

pt2 = pt.tac(elim_tac('disjE'))
print('pt2:', pt2)

pt: ProofTerm(P ∨ Q ⊢ Q ∨ P)
Gaps: P ∨ Q ⊢ Q ∨ P
pt2: ProofTerm(P ∨ Q ⊢ Q ∨ P)
Gaps: P ∨ Q ⊢ P ⟶ Q ∨ P
      P ∨ Q ⊢ Q ⟶ Q ∨ P


We can now finish the proof of this theorem using tactics we have already implemented:

In [23]:
print(pt.tacs(
    elim_tac('disjE'),
    intros_tac, rule_tac('disjI2'), assumption(),
    intros_tac, rule_tac('disjI1'), assumption()
))

ProofTerm(P ∨ Q ⊢ Q ∨ P)


Elimination can also be used on conjunctive assumptions. Here we used the theorem `conjE`:

In [24]:
theory.print_theorem('conjE')

conjE: ⊢ A ∧ B ⟶ (A ⟶ B ⟶ C) ⟶ C


After applying `elim` on `conjE`, a goal with $A$ and $B$ as additional assumptions is generated. Two applications of `intro_forall_tac` moves them to the hypothesis. This can be packaged up as a tactic as follows:

In [25]:
class conj_elim_tac(Tactic):
    def get_proof_term(self, goal):
        return ProofTerm.sorry(goal).tacs(
            elim_tac('conjE'), intro_imp_tac(), intro_imp_tac())

Let's try it on commutativity of conjunction from above:

In [26]:
P, Q = BoolVars('P Q')
goal = Implies(And(P, Q), And(Q, P))
pt = ProofTerm.sorry(Thm([], goal))
print(pt.tacs(intros_tac, rule_tac('conjI'),
              conj_elim_tac(), assumption(),
              conj_elim_tac(), assumption()))

ProofTerm(⊢ P ∧ Q ⟶ Q ∧ P)
